In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('./src') 
from xgboost_utils import train_xgb
from xgboost_predictor import generate_sparse_pgv

## Step 1: Train XGBoost Models per Receiver and Generate Sparse Maps

1- We train one XGBoost model per receiver  


In [ ]:
# load input and output data for receiver
receiver_idx         = 139
receiver_database    = np.load(f'/media/wolf6819/Elements/reciprocal_db/receiver_{receiver_idx}_database.npz')
inputs  = receiver_database['inputs']   # source coords and mechanism ([distance, azimuth, depth, strike, dip, rake, radiation, takeoff] )
outputs = receiver_database['outputs']  # pgv along two components (East-West, North-South) in mm/s

# train xgboost model 
pgv_true, pgv_pred = train_xgb(inputs, np.log(outputs[:,0]))  
fig, ax = plt.subplots(figsize=(5,5))
ax.scatter(pgv_true, pgv_pred)
ax.set_xlim([-3,5])
ax.set_ylim([-3,5])
ax.plot([-3, 5], [-3, 5], 'k--')
ax.set_xlabel('True log(PGV) (mm/s)')
ax.set_ylabel('Predicted log(PGV) (mm/s)')

2- We repeat the process above for all the receivers. The trained models for each receiver 
and for each of the components are stored in the `models/` directory. 

3 - We use the trained XGBoost models to generate sparse PGV maps for the forward simulations (used as input for Step 2) 

In [ ]:
# generate a sparse PGV map for input source coordinates and focal mechanism
# we show an example for the database with 50 source locations, and 50 mechanims per locations refered to as 50_50

data_tag   = '50_50'  
models_dir = '/media/wolf6819/Elements/models/'
spacing_km = 4 
station_coords_path = 'data/station_coords_sparse.npz' 
output_path         = 'data/step1_preds'

sparse_maps = generate_sparse_pgv(station_coords_path, models_dir, output_path, data_tag, spacing_km)

In [ ]:
# an example of a sparse PGV map  
station_coords_file = 'data/station_coords_sparse.npz'  
station_coords = np.load(station_coords_file)['station_coords']
fig, ax = plt.subplots(figsize=(5, 5))
ax.scatter(station_coords[:,0]/1e3, station_coords[:,1]/1e3, c = sparse_maps[100,:,:,0], s=100, cmap='plasma')
# make it nicer
ax.set_xlabel('East Distance (km)') 
ax.set_ylabel('North Distance (km)')    

## Step 2: Train the Encoder-MLP on Sparse Maps

We use the predicted sparse maps to train an EncoderMLP network 


In [4]:
from types import SimpleNamespace
from encoderMLP_predictor import run_train  

# Manual config 
config = SimpleNamespace(
  
    mode              = "train",
    data_tag          = "50_50_x4",
    downsample_factor = 4,
    
    # additional defaults 
    data_dir="data/",
    transform_input=True,
    transform_output=True,
    inc_gradient=False,
    inc_distance=True,
    normalize_output=True,
    normalize_input=True,
    nx_patch=8,
    ny_patch=8,
    fourier_features=False,
    D=32,
    gamma=5.0,
    n_samp_pts_per_patch=512,
    split=0.8,
    batch_size=32,
    learning_rate=1e-3,
    loss_type="mse",
    optimizer="adamw",
    num_epochs=1, 
    enc_type="edsr",
    in_channels=5,
    out_channels=32,
    conv_kernel_size=3,
    attention_kernel_size=1,
    num_features=32,
    num_blocks=8,
    nf=32,
    activation="sine",
    pad_size=4,
    sigma=3.0,
    results_dir=None  
)

# Run training
run_train(config)


Run ID: 20250617_094624_b323a2 | Seed: 2098295474
Number of patches: 32000


Training Epochs: 100%|██████████| 1/1 [01:48<00:00, 108.41s/it]

Epoch 0: Train Loss 0.1411 | Valid Loss 0.1065
Configuration saved to ./results/20250617_094624_b323a2/config.yaml.


Next, we test the trained network on an independent set of test events, the results are stored in the relevant dir in the `./results` folder

In [8]:
import yaml
from encoderMLP_predictor import run_test

with open("./results/20250617_094624_b323a2/config.yaml", "r") as f:
    cfg_dict = yaml.safe_load(f)

config             = SimpleNamespace(**cfg_dict)
config.mode        = "test"
config.results_dir = "./results/20250617_094624_b323a2"

# Run testing
run_test(config)

Testing using results from: ./results/20250617_094624_b323a2


Reconstructing patches For Test Maps: 100%|██████████| 12500/12500 [00:58<00:00, 212.67it/s]


Test SSIM: 0.7375


We show an example of the true (simulated) vs predicted PGV maps 

In [ ]:
preds = np.load('./results/20250617_084729_9a179d/test_preds.npy')
gts   = np.load('./results/20250617_084729_9a179d/test_gts.npy')  

fig, ax = plt.subplots(1, 2, figsize=(10, 5))  
# choose a random idx between 0 and the number of samples in preds  
idx  = np.random.randint(0, preds.shape[0]) 
pred = preds[idx,:,:,0]/10  # predicted PGV    
gt   = gts[idx,:,:,0]/10    # true PGV
norm = plt.Normalize(vmin=0, vmax=np.max(gt)) 
   
cf0 = ax[0].contourf(gt, cmap='plasma', norm=norm)
cf1 = ax[1].contourf(pred, cmap='plasma', norm=norm)

ax[0].set_title('True PGV')    
ax[1].set_title('Predicted PGV')
ax[0].set_xlabel('East Distance (km)')  
ax[0].set_ylabel('North Distance (km)') 
ax[1].set_xlabel('East Distance (km)')  

fig.subplots_adjust(right=0.85)  
cbar_ax = fig.add_axes([0.88, 0.15, 0.02, 0.7]) 
fig.colorbar(cf1, cax=cbar_ax).set_label("PGV (cm/s)", fontsize=14)

plt.show()